In [1]:
import torch.nn as nn
import torch

import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
import shutil
import pickle
import random
from tqdm import tqdm

In [2]:
# run_path = '/home/harishbabu/projects/PIPNet/runs/010-CUB-27-imgnet_OOD_cnext26_img=224_nprotos=20'
# run_path = '/home/harishbabu/projects/PIPNet/runs/031-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = '/home/harishbabu/projects/PIPNet/runs/032-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/035-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/043-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = "/home/harishbabu/projects/PIPNet/runs/036-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel_uniformity"
# run_path = "/home/harishbabu/projects/PIPNet/runs/041-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/042-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/044-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20-or-4per-desc_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/046-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=10per-desc_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/047-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc"
# run_path = "/home/harishbabu/projects/PIPNet/runs/048-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc_unit-sphere"
# run_path = "/home/harishbabu/projects/PIPNet/runs/051-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/052-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/055-CUB-18_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/056-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/057-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/058-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"

# with unit sphere
# run_path = "/home/harishbabu/projects/PIPNet/runs/059-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# unit sphere with softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/065-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# original hpipnet with 20 protos per node no KO, no OOD, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/062-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD"

# original hpipnet with 20 protos per node no KO, no OOD, WITH tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/063-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD_tanh-desc"

# with unit sphere but no AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/066-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_no-align_no-uni_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc, INCORRECT
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-incorrect-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, no softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/068-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# 071 with bias
# run_path = "/home/harishbabu/projects/PIPNet/runs/071-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_with-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 072 gumbel softmax, tau=0.5
# run_path = "/home/harishbabu/projects/PIPNet/runs/072-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 074 multiply_cs_softmax
run_path = "/home/harishbabu/projects/PIPNet/runs/074-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_multi-cs-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

try:
    sys.path.remove('/home/harishbabu/projects/PIPNet')
except:
    pass
sys.path.insert(0, os.path.join(run_path, 'source_clone'))


In [3]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
from util.node import Node
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
from util.func import get_patch_size
from util.data import ModifiedLabelLoader

Heatmaps showing where a prototype is found will not be generated because OpenCV is not installed.


In [4]:
from PIL import Image
import numpy as np
import pdb

def get_heatmap(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)


def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

## Load Model

In [5]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        if '031' in run_path: # this run uses a different phylogeny file that had an extra root node which is a mistake
            root = construct_phylo_tree('/home/harishbabu/data/phlyogenyCUB/18Species-with-extra-root-node/1_tree-consensus-Hacket-18Species-modified_cub-names_v1.phy')
        else:
            root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

if ('num_protos_per_descendant' in args) and (args.num_protos_per_descendant > 0):
    for node in root.nodes_with_children():
        node.set_num_protos(args.num_protos_per_descendant)

------------------------- No discretization -------------------------


In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

# ckpt_file_name = 'net_overspecific_pruned_replaced_thresh=0.5_last'
ckpt_file_name = 'net_trained_last'
# ckpt_file_name = 'net_trained_10'
# ckpt_file_name = 'net_pretrained'
epoch = ckpt_file_name.split('_')[-1]

ckpt_path = os.path.join(run_path, 'checkpoints', ckpt_file_name)
checkpoint = torch.load(ckpt_path, map_location=device)

if ckpt_file_name != 'net_trained_last':
    print('\n', (10*'-')+'WARNING: Not using the final trained model'+(10*'-'), '\n')

# Obtain the dataset and dataloaders
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)
if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
net.eval()
criterion = nn.NLLLoss(reduction='mean').to(device)

# Forward one batch through the backbone to get the latent output size
# with torch.no_grad():
#     xs1, _, _ = next(iter(trainloader))
#     xs1 = xs1.to(device)
#     proto_features, _, _ = net(xs1)
#     wshape = proto_features['root'].shape[-1]
#     args.wshape = wshape #needed for calculating image patch size
#     print("Output shape: ", proto_features['root'].shape, flush=True)
    
args.wshape = 26

Dropping 0 samples from trainloader
Dropping 0 samples from trainloader_pretraining
Dropping 0 samples from trainloader_normal
Dropping 0 samples from trainloader_normal_augment
Num classes (k) =  18 ['cub_001_Black_footed_Albatross', 'cub_002_Laysan_Albatross', 'cub_003_Sooty_Albatross', 'cub_004_Groove_billed_Ani', 'cub_023_Brandt_Cormorant'] etc.
Classes:  {'cub_001_Black_footed_Albatross': 0, 'cub_002_Laysan_Albatross': 1, 'cub_003_Sooty_Albatross': 2, 'cub_004_Groove_billed_Ani': 3, 'cub_023_Brandt_Cormorant': 4, 'cub_024_Red_faced_Cormorant': 5, 'cub_025_Pelagic_Cormorant': 6, 'cub_031_Black_billed_Cuckoo': 7, 'cub_032_Mangrove_Cuckoo': 8, 'cub_033_Yellow_billed_Cuckoo': 9, 'cub_045_Northern_Fulmar': 10, 'cub_050_Eared_Grebe': 11, 'cub_051_Horned_Grebe': 12, 'cub_052_Pied_billed_Grebe': 13, 'cub_053_Western_Grebe': 14, 'cub_086_Pacific_Loon': 15, 'cub_100_Brown_Pelican': 16, 'cub_101_White_Pelican': 17}


/home/harishbabu/.conda/envs/hpnet1/lib/python3.8/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Number of prototypes:  20
----------Prototypes per descendant: 4----------
Assigned 136 protos to node root
Assigned 24 protos to node 052+053
Assigned 104 protos to node 004+086
Assigned 16 protos to node 053+050
Assigned 24 protos to node 004+032
Assigned 72 protos to node 086+045
Assigned 8 protos to node 050+051
Assigned 16 protos to node 032+033
Assigned 64 protos to node 045+101
Assigned 8 protos to node 033+031
Assigned 24 protos to node 045+003
Assigned 32 protos to node 101+023
Assigned 16 protos to node 003+002
Assigned 8 protos to node 101+100
Assigned 16 protos to node 023+025
Assigned 8 protos to node 002+001
Assigned 8 protos to node 025+024


# Proto activations on leaf descendents - topk images using either NAIVE-HPIPNET or UNIT-SPACE-PROTOPOOL with HEATMAP w/o INTERPOLATION with SOFTMAX SCORE and COSINE SIMILARITY (for experiment 074)

In [7]:
import torch.nn.functional as F
import pdb

from pipnet.pipnet import functional_UnitConv2D

# specifically written for for experiment 074
def custom_forward(net, xs, inference):
    features = net.module._net(xs) 
    proto_features = {}
    # inner product between normalized unit vectors (cosine similarity)
    proto_features_inner_product = {}
    pooled = {}
    pooled_inner_product = {}
    out = {}
    for node in net.module.root.nodes_with_children():
        proto_features[node.name] = getattr(net.module, '_'+node.name+'_add_on')(features)
        proto_features[node.name] = net.module._softmax(proto_features[node.name])
        
        # getting inner product with kernel and input as unit vectors
        add_on = getattr(net.module, '_'+node.name+'_add_on')
        normalized_weight = F.normalize(add_on.weight.data, p=2, dim=(1, 2, 3)) # Normalize the kernels to unit vectors
        normalized_input = F.normalize(features, p=2, dim=1) # Normalize the input to unit vectors
        proto_features_inner_product[node.name] = F.conv2d(normalized_input, normalized_weight, bias=None)

        pooled[node.name] = net.module._pool(proto_features[node.name])
        if inference:
            pooled[node.name] = torch.where(pooled[node.name] < 0.1, 0., pooled[node.name])  #during inference, ignore all prototypes that have 0.1 similarity or lower
        out[node.name] = getattr(net.module, '_'+node.name+'_classification')(pooled[node.name]) #shape (bs*2, num_classes)
        
        # finding correspoding value to pooled in inner product
#         pdb.set_trace()
        output, indices = F.max_pool2d(proto_features[node.name], kernel_size=(26, 26), return_indices=True)# these are logits
        tensor_flattened = proto_features_inner_product[node.name].view(proto_features_inner_product[node.name].shape[0],\
                                                                        proto_features_inner_product[node.name].shape[1], -1)
        indices_flattened = indices.view(proto_features_inner_product[node.name].shape[0],\
                                         proto_features_inner_product[node.name].shape[1], -1)
        corresponding_values_in_proto_features_inner_product = torch.gather(tensor_flattened, 2, indices_flattened)
        corresponding_values_in_proto_features_inner_product = corresponding_values_in_proto_features_inner_product.view(proto_features_inner_product[node.name].shape[0],\
                                         proto_features_inner_product[node.name].shape[1], 1, 1)
        pooled_inner_product[node.name] = corresponding_values_in_proto_features_inner_product

    return features, proto_features, pooled, pooled_inner_product, out

def findCorrespondingToMax(base, target):
    output, indices = F.max_pool2d(base, kernel_size=(26, 26), return_indices=True)# these are logits
    tensor_flattened = target.view(target.shape[0], target.shape[1], -1)
    indices_flattened = indices.view(target.shape[0], target.shape[1], -1)
    corresponding_values_in_target = torch.gather(tensor_flattened, 2, indices_flattened)
    corresponding_values_in_target = corresponding_values_in_target.view(target.shape[0],\
                                     target.shape[1], 1, 1)
    pooled_target = corresponding_values_in_target
    return pooled_target

def customForwardWithCSandSoftmax(net, xs,  inference=False):
    features = net.module._net(xs) 
    proto_features = {}
    proto_features_cs = {}
    proto_features_softmaxed = {}
    pooled = {}
    pooled_cs = {}
    pooled_softmaxed = {}
    out = {}
    for node in net.module.root.nodes_with_children():
        proto_features[node.name] = getattr(net.module, '_'+node.name+'_add_on')(features)

        if net.module.args.softmax == 'y':
            proto_features_softmaxed[node.name] = net.module._softmax(proto_features[node.name])
            proto_features[node.name] = proto_features[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'
        elif net.module.args.gumbel_softmax == 'y':
            proto_features_softmaxed[node.name] = net.module._gumbel_softmax(proto_features[node.name])
            proto_features[node.name] = proto_features[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'

        if net.module.args.multiply_cs_softmax == 'y':
            prototypes = getattr(net.module, '_'+node.name+'_add_on')
            proto_features_cs[node.name] = functional_UnitConv2D(features, prototypes.weight, prototypes.bias)
            proto_features[node.name] = proto_features_cs[node.name] * proto_features_softmaxed[node.name]

        pooled[node.name] = net.module._pool(proto_features[node.name])

        pooled_cs[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_cs[node.name])

        pooled_softmaxed[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_softmaxed[node.name])

        if inference:
            pooled[node.name] = torch.where(pooled[node.name] < 0.1, 0., pooled[node.name])  #during inference, ignore all prototypes that have 0.1 similarity or lower
        out[node.name] = getattr(net.module, '_'+node.name+'_classification')(pooled[node.name]) #shape (bs*2, num_classes) # these are logits

    return features, proto_features, pooled, pooled_cs, pooled_softmaxed, out


# Proto activations on leaf descendents - topk images

from util.data import ModifiedLabelLoader
from collections import defaultdict
import heapq
import pdb
from util.vis_pipnet import get_img_coordinates
import torchvision.transforms as transforms
from PIL import ImageFont, Image, ImageDraw as D
import torchvision

topk = 6
save_images = True
font = ImageFont.truetype("arial.ttf", 50)

def get_heap():
    list_ = []
    heapq.heapify(list_)
    return list_

patchsize, skip = get_patch_size(args)

for node in root.nodes_with_children():
#     if node.name == 'root':
#         continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

    name2label = testloader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(testloader, node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}
    
    imgs = modifiedLabelLoader.filtered_imgs

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> list of top-k activations
    proto_mean_activations = defaultdict(lambda: defaultdict(get_heap))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
        if coarse_label2name[ys.item()] not in non_leaf_children_names:
            continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            model_output = customForwardWithCSandSoftmax(net, xs, inference=False)
            _, softmaxes, pooled, pooled_cs, pooled_softmaxed, _ = model_output
#             if len(model_output) == 4:
#                 softmaxes, pooled, pooled_ip, _ = model_output
#             elif len(model_output) == 5:
#                 _, softmaxes, pooled, pooled_ip, _ = model_output
            pooled = pooled[node.name].squeeze(0) 
#             pdb.set_trace()
            pooled_cs = pooled_cs[node.name].squeeze()
            pooled_softmaxed = pooled_softmaxed[node.name].squeeze()
            softmaxes = softmaxes[node.name]#.squeeze(0)

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]
                
                # Take the max per prototype.                             
                max_per_prototype, max_idx_per_prototype = torch.max(softmaxes, dim=0)
                max_per_prototype_h, max_idx_per_prototype_h = torch.max(max_per_prototype, dim=1)
                max_per_prototype_w, max_idx_per_prototype_w = torch.max(max_per_prototype_h, dim=1) #shape (num_prototypes)
                
                h_idx = max_idx_per_prototype_h[p, max_idx_per_prototype_w[p]]
                w_idx = max_idx_per_prototype_w[p]

                if len(relevant_proto_class_names) == 0:
                    continue
                
                if (len(relevant_proto_class_names) == 1) and (relevant_proto_class_names[0] not in non_leaf_children_names):
                    continue
                
                h_coor_min, h_coor_max, w_coor_min, w_coor_max = get_img_coordinates(args.image_size, softmaxes.shape, patchsize, skip, h_idx, w_idx)
                latent_activation = softmaxes[:, p, :, :]
                if (coarse_label2name[ys.item()] in relevant_proto_class_names):
                    child_node = root.get_node(coarse_label2name[ys.item()])
                    leaf_descendent = label2name[orig_y.item()][4:7]
                    img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                    if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                        heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                          (pooled[p].item(), pooled_cs[p].item(), pooled_softmaxed[p].item(), img_to_open,\
                                           (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                    else:
                        heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                       (pooled[p].item(), pooled_cs[p].item(), pooled_softmaxed[p].item(), img_to_open,\
                                        (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))

                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)

    
    print('Node', node.name)
    for child_classname in class_and_prototypes:
        
        print('\t'*1, 'Child:', child_classname)
        for p in class_and_prototypes[child_classname]:
            
            logstr = '\t'*2 + f'Proto:{p} '
            for leaf_descendent in proto_mean_activations[p]:
                mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
                num_images = len(proto_mean_activations[p][leaf_descendent])
                logstr += f'{leaf_descendent}:({mean_activation}) '
            print(logstr)
            
            if save_images:
                patches = []
                right_descriptions = []
                text_region_width = 3 # 3x the width of a patch
                for leaf_descendent, heap in proto_mean_activations[p].items():
                    heap = sorted(heap)[::-1]
                    mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    mean_cosine_similarity = round(np.mean([activation_cs for _, activation_cs, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    mean_softmax = round(np.mean([activation_softmax for _, _, activation_softmax, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    
                    for ele in heap:
                        activation, activation_cs, activation_softmax, img_to_open, (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation = ele
                        image = transforms.Resize(size=(args.image_size, args.image_size))(Image.open(img_to_open))
                        img_tensor = transforms.ToTensor()(image)#.unsqueeze_(0) #shape (1, 3, h, w)
                        img_tensor_patch = img_tensor[:, h_coor_min:h_coor_max, w_coor_min:w_coor_max]
                        overlayed_image_np = get_heatmap_uninterpolated(latent_activation, img_tensor)
                        overlayed_image = torch.tensor(overlayed_image_np).permute(2, 0, 1).float() / 255.
                        patches.append(overlayed_image)

                    # description on the right hand side
                    text = f'{mean_softmax}, {mean_cosine_similarity}, {mean_activation}, {leaf_descendent}'
                    txtimage = Image.new("RGB", (patches[0].shape[-2]*text_region_width,patches[0].shape[-1]), (0, 0, 0))
                    draw = D.Draw(txtimage)
                    draw.text((300, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                    txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)
                    right_descriptions.append(txttensor)

                grid = torchvision.utils.make_grid(patches, nrow=topk, padding=1)
                grid_right_descriptions = torchvision.utils.make_grid(right_descriptions, nrow=1, padding=1)

                # merging right description with the grid of images
                grid = torch.cat([grid, grid_right_descriptions], dim=-1)

                # description on the top
                text = f'Node:{node.name}, p{p}, Child:{child_classname}'
                txtimage = Image.new("RGB", (grid.shape[-1], 224), (0, 0, 0))
                draw = D.Draw(txtimage)
                draw.text((200, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)

                # merging top description with the grid of images
                grid = torch.cat([grid, txttensor], dim=1)

                os.makedirs(os.path.join(run_path, f'descendent_specific_topk_heatmap_uninterp_cosine_and_softmax_testloader_ep={epoch}', node.name), exist_ok=True)
                torchvision.utils.save_image(grid, os.path.join(run_path, f'descendent_specific_topk_heatmap_uninterp_cosine_and_softmax_testloader_ep={epoch}', node.name, f'{child_classname}-p{p}.png'))

print('Done !!!')

/tmp/ipykernel_96780/3535226083.py:35: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  resample=Image.NEAREST ))


Node root
	 Child: 004+086
		Proto:0 001:(0.004) 002:(0.0024) 003:(0.0027) 004:(0.0021) 023:(0.0027) 024:(0.0023) 025:(0.0019) 031:(0.0037) 032:(0.0021) 033:(0.0034) 045:(0.0026) 086:(0.0028) 100:(0.0018) 101:(0.0018) 
		Proto:1 001:(0.0011) 002:(0.0011) 003:(0.0012) 004:(0.0027) 023:(0.0029) 024:(0.0033) 025:(0.0028) 031:(0.0023) 032:(0.0018) 033:(0.0013) 045:(0.0008) 086:(0.0006) 100:(0.0015) 101:(0.0008) 
		Proto:2 001:(0.4898) 002:(0.5064) 003:(0.5492) 004:(0.2951) 023:(0.0176) 024:(0.1426) 025:(0.0377) 031:(0.477) 032:(0.3838) 033:(0.4694) 045:(0.4916) 086:(0.0719) 100:(0.414) 101:(0.4485) 
		Proto:3 001:(0.0015) 002:(0.0015) 003:(0.0016) 004:(0.0017) 023:(0.0013) 024:(0.001) 025:(0.0012) 031:(0.0022) 032:(0.0022) 033:(0.002) 045:(0.002) 086:(0.0014) 100:(0.0013) 101:(0.0014) 
		Proto:4 001:(0.001) 002:(0.0011) 003:(0.001) 004:(0.0012) 023:(0.0011) 024:(0.0011) 025:(0.0011) 031:(0.0012) 032:(0.001) 033:(0.0011) 045:(0.001) 086:(0.0008) 100:(0.0009) 101:(0.0008) 
		Proto:128 001:(0

		Proto:63 001:(0.0067) 002:(0.0076) 003:(0.0075) 004:(0.0075) 023:(0.0072) 024:(0.0072) 025:(0.0082) 031:(0.0061) 032:(0.0085) 033:(0.0106) 045:(0.0031) 086:(0.0022) 100:(0.0127) 101:(0.0033) 
		Proto:64 001:(0.0458) 002:(0.0234) 003:(0.0158) 004:(0.2555) 023:(0.3144) 024:(0.4249) 025:(0.2936) 031:(0.0068) 032:(0.0571) 033:(0.0623) 045:(0.0051) 086:(0.003) 100:(0.0455) 101:(0.0019) 
		Proto:65 001:(0.0011) 002:(0.001) 003:(0.0012) 004:(0.0009) 023:(0.0009) 024:(0.0009) 025:(0.0009) 031:(0.001) 032:(0.0017) 033:(0.001) 045:(0.0011) 086:(0.001) 100:(0.0009) 101:(0.0013) 
		Proto:66 001:(0.1329) 002:(0.1116) 003:(0.1028) 004:(0.2048) 023:(0.0442) 024:(0.0754) 025:(0.1517) 031:(0.2463) 032:(0.2512) 033:(0.2514) 045:(0.0588) 086:(0.0052) 100:(0.3072) 101:(0.3644) 
		Proto:69 001:(0.3264) 002:(0.2926) 003:(0.3696) 004:(0.4716) 023:(0.2299) 024:(0.36) 025:(0.3494) 031:(0.4692) 032:(0.4064) 033:(0.488) 045:(0.2729) 086:(0.0372) 100:(0.3018) 101:(0.1936) 
		Proto:70 001:(0.0009) 002:(0.0008) 0

Node 052+053
	 Child: 053+050
		Proto:2 050:(0.5218) 051:(0.4555) 053:(0.4458) 
		Proto:6 050:(0.4848) 051:(0.4191) 053:(0.1902) 
		Proto:10 050:(0.0162) 051:(0.0312) 053:(0.4592) 
		Proto:15 050:(0.5941) 051:(0.5771) 053:(0.659) 
		Proto:16 050:(0.4263) 051:(0.3663) 053:(0.4272) 
		Proto:18 050:(0.3311) 051:(0.4146) 053:(0.4237) 
		Proto:22 050:(0.5118) 051:(0.5005) 053:(0.5007) 


Node 004+086
	 Child: 086+045
		Proto:5 001:(0.3103) 002:(0.2379) 003:(0.3405) 023:(0.2663) 024:(0.2549) 025:(0.2414) 045:(0.2739) 086:(0.2774) 100:(0.1571) 101:(0.213) 
		Proto:7 001:(0.2097) 002:(0.2853) 003:(0.2992) 023:(0.4347) 024:(0.3932) 025:(0.4108) 045:(0.1839) 086:(0.437) 100:(0.4141) 101:(0.3915) 
		Proto:8 001:(0.337) 002:(0.1852) 003:(0.1847) 023:(0.4057) 024:(0.0902) 025:(0.2938) 045:(0.354) 086:(0.481) 100:(0.3505) 101:(0.3434) 
		Proto:9 001:(0.0524) 002:(0.0081) 003:(0.0089) 023:(0.0303) 024:(0.0043) 025:(0.0193) 045:(0.0297) 086:(0.045) 100:(0.024) 101:(0.0081) 
		Proto:10 001:(0.4254) 002:(0.4574) 003:(0.4603) 023:(0.279) 024:(0.3487) 025:(0.2203) 045:(0.1063) 086:(0.2892) 100:(0.3702) 101:(0.3137) 
		Proto:11 001:(0.4343) 002:(0.4391) 003:(0.3597) 023:(0.3223) 024:(0.2702) 025:(0.1594) 045:(0.0297) 086:(0.2858) 100:(0.3975) 101:(0.3432) 
		Proto:20 001:(0.0035) 002:(0.0019) 003:(0.0019) 023:(0.0038) 024:(0.0017) 025:(0.002) 045:(0.0037) 086:(0.0045) 100:(0.0022) 101

...

Node 045+101
	 Child: 101+023
		Proto:3 023:(0.3701) 024:(0.3427) 025:(0.2778) 100:(0.5149) 101:(0.288) 
		Proto:37 023:(0.0183) 024:(0.0044) 025:(0.0097) 100:(0.0126) 101:(0.0061) 
		Proto:6 023:(0.4151) 024:(0.3909) 025:(0.4339) 100:(0.3824) 101:(0.0097) 
		Proto:7 023:(0.4544) 024:(0.4923) 025:(0.4803) 100:(0.4969) 101:(0.4496) 
		Proto:8 023:(0.0082) 024:(0.0123) 025:(0.0113) 100:(0.012) 101:(0.0079) 
		Proto:9 023:(0.3241) 024:(0.3232) 025:(0.3171) 100:(0.3304) 101:(0.4055) 
		Proto:41 023:(0.2953) 024:(0.2444) 025:(0.1369) 100:(0.377) 101:(0.3986) 
		Proto:59 023:(0.4559) 024:(0.4719) 025:(0.4924) 100:(0.3213) 101:(0.2944) 
		Proto:14 023:(0.2668) 024:(0.2894) 025:(0.3106) 100:(0.1134) 101:(0.0198) 
		Proto:48 023:(0.2118) 024:(0.1611) 025:(0.2557) 100:(0.278) 101:(0.1981) 
		Proto:18 023:(0.441) 024:(0.4066) 025:(0.4365) 100:(0.415) 101:(0.3706) 
		Proto:19 023:(0.384) 024:(0.3278) 025:(0.4315) 100:(0.4914) 101:(0.4532) 
		Proto:51 023:(0.2951) 024:(0.2314) 025:(0.3017) 100:(0.4

Node 045+003
	 Child: 003+002
		Proto:5 001:(0.5214) 002:(0.5277) 003:(0.4371) 
		Proto:6 001:(0.3393) 002:(0.2839) 003:(0.2456) 
		Proto:7 001:(0.5713) 002:(0.4519) 003:(0.1698) 
		Proto:9 001:(0.4411) 002:(0.658) 003:(0.5329) 
		Proto:10 001:(0.5803) 002:(0.6444) 003:(0.6084) 
		Proto:12 001:(0.4978) 002:(0.0133) 003:(0.0109) 
		Proto:15 001:(0.4334) 002:(0.4403) 003:(0.45) 
		Proto:17 001:(0.5505) 002:(0.5216) 003:(0.5846) 
		Proto:19 001:(0.623) 002:(0.6028) 003:(0.567) 
		Proto:21 001:(0.4637) 002:(0.4713) 003:(0.4801) 
		Proto:22 001:(0.4282) 002:(0.4525) 003:(0.2733) 


Node 101+023
	 Child: 023+025
		Proto:0 023:(0.5316) 024:(0.5492) 025:(0.5698) 
		Proto:3 023:(0.5084) 024:(0.4597) 025:(0.5161) 
		Proto:4 023:(0.4423) 024:(0.4673) 025:(0.4698) 
		Proto:6 023:(0.3737) 024:(0.3824) 025:(0.3238) 
		Proto:12 023:(0.3687) 024:(0.3835) 025:(0.4526) 
		Proto:13 023:(0.037) 024:(0.3079) 025:(0.0497) 
		Proto:14 023:(0.42) 024:(0.3654) 025:(0.4376) 
		Proto:17 023:(0.5212) 024:(0.452) 025:(0.5035) 
		Proto:20 023:(0.4371) 024:(0.3043) 025:(0.3064) 
		Proto:22 023:(0.3068) 024:(0.3184) 025:(0.2965) 
		Proto:27 023:(0.3807) 024:(0.3448) 025:(0.4086) 
		Proto:31 023:(0.483) 024:(0.4578) 025:(0.4719) 
	 Child: 101+100
		Proto:1 100:(0.5787) 101:(0.5605) 
		Proto:2 100:(0.5724) 101:(0.553) 
		Proto:9 100:(0.4134) 101:(0.1247) 
		Proto:16 100:(0.6195) 101:(0.5887) 
		Proto:21 100:(0.3627) 101:(0.4418) 
		Proto:25 100:(0.6201) 101:(0.6747) 
		Proto:30 100:(0.6134) 101:(0.6072) 


Node 003+002
	 Child: 002+001
		Proto:0 001:(0.7191) 002:(0.7768) 
		Proto:4 001:(0.4682) 002:(0.4824) 
		Proto:5 001:(0.6094) 002:(0.3669) 
		Proto:7 001:(0.5762) 002:(0.5224) 
		Proto:8 001:(0.2077) 002:(0.1579) 
		Proto:10 001:(0.5885) 002:(0.5717) 
		Proto:11 001:(0.4383) 002:(0.3043) 
		Proto:13 001:(0.7415) 002:(0.659) 
		Proto:14 001:(0.4285) 002:(0.3636) 
		Proto:15 001:(0.4155) 002:(0.2445) 


Node 023+025
	 Child: 025+024
		Proto:0 024:(0.5298) 025:(0.5154) 
		Proto:3 024:(0.139) 025:(0.1895) 
		Proto:4 024:(0.4726) 025:(0.4431) 
		Proto:5 024:(0.6065) 025:(0.4046) 
		Proto:6 024:(0.6194) 025:(0.5335) 
		Proto:9 024:(0.5334) 025:(0.5387) 
		Proto:11 024:(0.4281) 025:(0.4038) 
Done !!!


# Proto activations on NON leaf descendents - topk images using either NAIVE-HPIPNET or UNIT-SPACE-PROTOPOOL with HEATMAP w/o INTERPOLATION with SOFTMAX SCORE and COSINE SIMILARITY (for experiment 074)

In [8]:
import torch.nn.functional as F
import pdb

from pipnet.pipnet import functional_UnitConv2D

# specifically written for for experiment 074
def custom_forward(net, xs, inference):
    features = net.module._net(xs) 
    proto_features = {}
    # inner product between normalized unit vectors (cosine similarity)
    proto_features_inner_product = {}
    pooled = {}
    pooled_inner_product = {}
    out = {}
    for node in net.module.root.nodes_with_children():
        proto_features[node.name] = getattr(net.module, '_'+node.name+'_add_on')(features)
        proto_features[node.name] = net.module._softmax(proto_features[node.name])
        
        # getting inner product with kernel and input as unit vectors
        add_on = getattr(net.module, '_'+node.name+'_add_on')
        normalized_weight = F.normalize(add_on.weight.data, p=2, dim=(1, 2, 3)) # Normalize the kernels to unit vectors
        normalized_input = F.normalize(features, p=2, dim=1) # Normalize the input to unit vectors
        proto_features_inner_product[node.name] = F.conv2d(normalized_input, normalized_weight, bias=None)

        pooled[node.name] = net.module._pool(proto_features[node.name])
        if inference:
            pooled[node.name] = torch.where(pooled[node.name] < 0.1, 0., pooled[node.name])  #during inference, ignore all prototypes that have 0.1 similarity or lower
        out[node.name] = getattr(net.module, '_'+node.name+'_classification')(pooled[node.name]) #shape (bs*2, num_classes)
        
        # finding correspoding value to pooled in inner product
#         pdb.set_trace()
        output, indices = F.max_pool2d(proto_features[node.name], kernel_size=(26, 26), return_indices=True)# these are logits
        tensor_flattened = proto_features_inner_product[node.name].view(proto_features_inner_product[node.name].shape[0],\
                                                                        proto_features_inner_product[node.name].shape[1], -1)
        indices_flattened = indices.view(proto_features_inner_product[node.name].shape[0],\
                                         proto_features_inner_product[node.name].shape[1], -1)
        corresponding_values_in_proto_features_inner_product = torch.gather(tensor_flattened, 2, indices_flattened)
        corresponding_values_in_proto_features_inner_product = corresponding_values_in_proto_features_inner_product.view(proto_features_inner_product[node.name].shape[0],\
                                         proto_features_inner_product[node.name].shape[1], 1, 1)
        pooled_inner_product[node.name] = corresponding_values_in_proto_features_inner_product

    return features, proto_features, pooled, pooled_inner_product, out

def findCorrespondingToMax(base, target):
    output, indices = F.max_pool2d(base, kernel_size=(26, 26), return_indices=True)# these are logits
    tensor_flattened = target.view(target.shape[0], target.shape[1], -1)
    indices_flattened = indices.view(target.shape[0], target.shape[1], -1)
    corresponding_values_in_target = torch.gather(tensor_flattened, 2, indices_flattened)
    corresponding_values_in_target = corresponding_values_in_target.view(target.shape[0],\
                                     target.shape[1], 1, 1)
    pooled_target = corresponding_values_in_target
    return pooled_target

def customForwardWithCSandSoftmax(net, xs,  inference=False):
    features = net.module._net(xs) 
    proto_features = {}
    proto_features_cs = {}
    proto_features_softmaxed = {}
    pooled = {}
    pooled_cs = {}
    pooled_softmaxed = {}
    out = {}
    for node in net.module.root.nodes_with_children():
        proto_features[node.name] = getattr(net.module, '_'+node.name+'_add_on')(features)

        if net.module.args.softmax == 'y':
            proto_features_softmaxed[node.name] = net.module._softmax(proto_features[node.name])
            proto_features[node.name] = proto_features[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'
        elif net.module.args.gumbel_softmax == 'y':
            proto_features_softmaxed[node.name] = net.module._gumbel_softmax(proto_features[node.name])
            proto_features[node.name] = proto_features[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'

        if net.module.args.multiply_cs_softmax == 'y':
            prototypes = getattr(net.module, '_'+node.name+'_add_on')
            proto_features_cs[node.name] = functional_UnitConv2D(features, prototypes.weight, prototypes.bias)
            proto_features[node.name] = proto_features_cs[node.name] * proto_features_softmaxed[node.name]

        pooled[node.name] = net.module._pool(proto_features[node.name])

        pooled_cs[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_cs[node.name])

        pooled_softmaxed[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_softmaxed[node.name])

        if inference:
            pooled[node.name] = torch.where(pooled[node.name] < 0.1, 0., pooled[node.name])  #during inference, ignore all prototypes that have 0.1 similarity or lower
        out[node.name] = getattr(net.module, '_'+node.name+'_classification')(pooled[node.name]) #shape (bs*2, num_classes) # these are logits

    return features, proto_features, pooled, pooled_cs, pooled_softmaxed, out


# Proto activations on leaf descendents - topk images

from util.data import ModifiedLabelLoader
from collections import defaultdict
import heapq
import pdb
from util.vis_pipnet import get_img_coordinates
import torchvision.transforms as transforms
from PIL import ImageFont, Image, ImageDraw as D
import torchvision

topk = 6
save_images = True
font = ImageFont.truetype("arial.ttf", 50)

def get_heap():
    list_ = []
    heapq.heapify(list_)
    return list_

patchsize, skip = get_patch_size(args)

for node in root.nodes_with_children():
#     if node.name == 'root':
#         continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

    name2label = testloader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(testloader, node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}
    
    imgs = modifiedLabelLoader.filtered_imgs

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> list of top-k activations
    proto_mean_activations = defaultdict(lambda: defaultdict(get_heap))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
        if coarse_label2name[ys.item()] not in non_leaf_children_names:
            continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            model_output = customForwardWithCSandSoftmax(net, xs, inference=False)
            _, softmaxes, pooled, pooled_cs, pooled_softmaxed, _ = model_output
#             if len(model_output) == 4:
#                 softmaxes, pooled, pooled_ip, _ = model_output
#             elif len(model_output) == 5:
#                 _, softmaxes, pooled, pooled_ip, _ = model_output
            pooled = pooled[node.name].squeeze(0) 
#             pdb.set_trace()
            pooled_cs = pooled_cs[node.name].squeeze()
            pooled_softmaxed = pooled_softmaxed[node.name].squeeze()
            softmaxes = softmaxes[node.name]#.squeeze(0)

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]
                
                # Take the max per prototype.                             
                max_per_prototype, max_idx_per_prototype = torch.max(softmaxes, dim=0)
                max_per_prototype_h, max_idx_per_prototype_h = torch.max(max_per_prototype, dim=1)
                max_per_prototype_w, max_idx_per_prototype_w = torch.max(max_per_prototype_h, dim=1) #shape (num_prototypes)
                
                h_idx = max_idx_per_prototype_h[p, max_idx_per_prototype_w[p]]
                w_idx = max_idx_per_prototype_w[p]

                if len(relevant_proto_class_names) == 0:
                    continue
                
                if (len(relevant_proto_class_names) == 1) and (relevant_proto_class_names[0] not in non_leaf_children_names):
                    continue
                
                h_coor_min, h_coor_max, w_coor_min, w_coor_max = get_img_coordinates(args.image_size, softmaxes.shape, patchsize, skip, h_idx, w_idx)
                latent_activation = softmaxes[:, p, :, :]
                if (coarse_label2name[ys.item()] not in relevant_proto_class_names):
                    child_node = root.get_node(coarse_label2name[ys.item()])
                    leaf_descendent = label2name[orig_y.item()][4:7]
                    img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                    if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                        heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                          (pooled[p].item(), pooled_cs[p].item(), pooled_softmaxed[p].item(), img_to_open,\
                                           (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                    else:
                        heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                       (pooled[p].item(), pooled_cs[p].item(), pooled_softmaxed[p].item(), img_to_open,\
                                        (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))

                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)

    
    print('Node', node.name)
    for child_classname in class_and_prototypes:
        
        print('\t'*1, 'Child:', child_classname)
        for p in class_and_prototypes[child_classname]:
            
            logstr = '\t'*2 + f'Proto:{p} '
            for leaf_descendent in proto_mean_activations[p]:
                mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
                num_images = len(proto_mean_activations[p][leaf_descendent])
                logstr += f'{leaf_descendent}:({mean_activation}) '
            print(logstr)
            
            if len(proto_mean_activations_non_descendants[p]) == 0:
                continue
            
            if save_images:
                patches = []
                right_descriptions = []
                text_region_width = 3 # 3x the width of a patch
                for leaf_descendent, heap in proto_mean_activations[p].items():
                    heap = sorted(heap)[::-1]
                    mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    mean_cosine_similarity = round(np.mean([activation_cs for _, activation_cs, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    mean_softmax = round(np.mean([activation_softmax for _, _, activation_softmax, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    
                    for ele in heap:
                        activation, activation_cs, activation_softmax, img_to_open, (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation = ele
                        image = transforms.Resize(size=(args.image_size, args.image_size))(Image.open(img_to_open))
                        img_tensor = transforms.ToTensor()(image)#.unsqueeze_(0) #shape (1, 3, h, w)
                        img_tensor_patch = img_tensor[:, h_coor_min:h_coor_max, w_coor_min:w_coor_max]
                        overlayed_image_np = get_heatmap_uninterpolated(latent_activation, img_tensor)
                        overlayed_image = torch.tensor(overlayed_image_np).permute(2, 0, 1).float() / 255.
                        patches.append(overlayed_image)

                    # description on the right hand side
                    text = f'{mean_softmax}, {mean_cosine_similarity}, {mean_activation}, {leaf_descendent}'
                    txtimage = Image.new("RGB", (patches[0].shape[-2]*text_region_width,patches[0].shape[-1]), (0, 0, 0))
                    draw = D.Draw(txtimage)
                    draw.text((300, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                    txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)
                    right_descriptions.append(txttensor)

                grid = torchvision.utils.make_grid(patches, nrow=topk, padding=1)
                grid_right_descriptions = torchvision.utils.make_grid(right_descriptions, nrow=1, padding=1)

                # merging right description with the grid of images
                grid = torch.cat([grid, grid_right_descriptions], dim=-1)

                # description on the top
                text = f'Node:{node.name}, p{p}, Child:{child_classname}'
                txtimage = Image.new("RGB", (grid.shape[-1], 224), (0, 0, 0))
                draw = D.Draw(txtimage)
                draw.text((200, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)

                # merging top description with the grid of images
                grid = torch.cat([grid, txttensor], dim=1)

                os.makedirs(os.path.join(run_path, f'non_descendent_specific_topk_heatmap_uninterp_cosine_and_softmax_testloader_ep={epoch}', node.name), exist_ok=True)
                torchvision.utils.save_image(grid, os.path.join(run_path, f'non_descendent_specific_topk_heatmap_uninterp_cosine_and_softmax_testloader_ep={epoch}', node.name, f'{child_classname}-p{p}.png'))

print('Done !!!')

/tmp/ipykernel_22056/3535226083.py:35: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  resample=Image.NEAREST ))


Node root
	 Child: 004+086
		Proto:0 050:(0.0014) 051:(0.0014) 052:(0.0012) 053:(0.0017) 
		Proto:1 050:(0.0007) 051:(0.0006) 052:(0.0006) 053:(0.0007) 
		Proto:2 050:(0.0557) 051:(0.0009) 052:(0.0016) 053:(0.0191) 
		Proto:3 050:(0.0013) 051:(0.0012) 052:(0.0008) 053:(0.0011) 
		Proto:4 050:(0.0007) 051:(0.0008) 052:(0.0009) 053:(0.0008) 
		Proto:128 050:(0.0014) 051:(0.0033) 052:(0.0008) 053:(0.0187) 
		Proto:6 050:(0.0299) 051:(0.011) 052:(0.0067) 053:(0.0027) 
		Proto:129 050:(0.0013) 051:(0.0009) 052:(0.001) 053:(0.001) 
		Proto:130 050:(0.0426) 051:(0.0252) 052:(0.0112) 053:(0.0244) 
		Proto:131 050:(0.0356) 051:(0.1034) 052:(0.0272) 053:(0.079) 
		Proto:10 050:(0.1116) 051:(0.0836) 052:(0.0688) 053:(0.0347) 
		Proto:132 050:(0.0011) 051:(0.0013) 052:(0.0006) 053:(0.001) 
		Proto:12 050:(0.0011) 051:(0.0011) 052:(0.001) 053:(0.0011) 
		Proto:13 050:(0.0031) 051:(0.0033) 052:(0.0025) 053:(0.0044) 
		Proto:14 050:(0.0105) 051:(0.0035) 052:(0.0016) 053:(0.003) 
		Proto:134 050:(0.00

Node 052+053
	 Child: 053+050
		Proto:2 


RuntimeError: stack expects a non-empty TensorList